# For Training and Loading the Pretrained Model on a Fresh Dataset

In [ ]:
import pandas as pd
import numpy as np
import os
from PIL import Image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
from keras.layers import Input
import tensorflow as tf
import tensorflow_hub as hub
# import tf.keras.callbacks 

In [ ]:
print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)

# Check for GPU
print("GPU", "available" if tf.config.list_physical_devices("GPU") else "not available")

TF version: 2.9.2
Hub version: 0.12.0
GPU available


In [ ]:
# # Running this cell will provide you with a token to link your drive to this notebook
from google.colab import drive
import sys

drive.mount('/content/gdrive/')
sys.path.append('/content/gdrive/My Drive/Datathon/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
%cd '/content/gdrive/My Drive/Datathon/'

/content/gdrive/.shortcut-targets-by-id/18Pjuiby86W8tPsgJuQAMo0AMjzsG0pLw/Datathon


In [ ]:
!ls

'~'				        mobile_net_l.png
 Datathon2.ipynb		        models
 datathonindoml-2022.zip	        models_archit
 densenet_a.png			       'New Models.ipynb'
 dense-net-logger.csv		        predicted_label_3.csv
 densenet_l.png			        predicted_label.csv
 drive				       'rekhani LSTM.ipynb'
 efficient_net_a.png		        resnet_a.png
 efficient-net-logger.csv	        res-net-logger.csv
 efficient_net_l.png		        resnet_l.png
'few shot.ipynb'		        sample_submission.csv
'for RESNETS.ipynb'		        train
 full_np_array.npy		        train_labels.csv
 kaggle-indoml-submission.csv	        validation
 kaggle-indoml-submission-model-2.csv   vision_trans_a.png
 logs				       'Vision Transformers.ipynb'
 mobile_net_a.png		        vision-trans-logger.csv
 mobile-net-logger.csv		        vision_trans_l.png
 mobile-net-logger.gsheet


In [ ]:
train_labels_csv = pd.read_csv("train_labels.csv")

In [ ]:
labels = train_labels_csv["label"].to_numpy() # convert labels column to NumPy array (from Training Dataset)
# Finding the unique labels
unique_labels = np.unique(labels)
# Turn every label into a boolean array
boolean_labels = [label == np.array(unique_labels) for label in labels]


In [ ]:
# Create pathnames from image ID's
train_path = "train/train/"
filenames = [train_path + str(fname) + ".jpeg" for fname in train_labels_csv["id"]]      # Fetching training files' IDs from train_labels_csv

val_path = "validation/validation/"
val_filenames = [val_path + str(fname) for fname in os.listdir(val_path)]       # Fetching Validation files' IDs from the validation set

In [ ]:
X_test = filenames[4000:5000]
y_test = boolean_labels[4000:5000]

In [ ]:
# Setup X & y variables
X = filenames[0:4000]
y = boolean_labels[0:4000]

In [ ]:
# Define image size
IMG_SIZE = 224

def process_image(image_path):
  """
  Takes an image file path and turns it into a Tensor.
  """
  # Read in image file
  image = tf.io.read_file(image_path)
  # Turn the jpeg image into numerical Tensor with 3 colour channels (Red, Green, Blue)
  image = tf.image.decode_jpeg(image, channels=3)
  # Convert the colour channel values from 0-225 values to 0-1 values
  image = tf.image.convert_image_dtype(image, tf.float32)
  # Resize the image to our desired size (224, 244)
  image = tf.image.resize(image, size=[IMG_SIZE, IMG_SIZE])
  return image

In [ ]:
# Define image size
IMG_SIZE = 256

def process_image_2(image_path):
  """
  Takes an image file path and turns it into a Tensor.
  """
  # Read in image file
  image = tf.io.read_file(image_path)
  # Turn the jpeg image into numerical Tensor with 3 colour channels (Red, Green, Blue)
  image = tf.image.decode_jpeg(image, channels=3)
  # Convert the colour channel values from 0-225 values to 0-1 values
  image = tf.image.convert_image_dtype(image, tf.float32)
  # Resize the image to our desired size (224, 244)
  image = tf.image.resize(image, size=[IMG_SIZE, IMG_SIZE])
  return image

In [ ]:
# Create a simple function to return a tuple (image, label)
def get_image_label(image_path, label):
  """
  Takes an image file path name and the associated label,
  processes the image and returns a tuple of (image, label).
  """
  image = process_image(image_path)
  return image, label

In [ ]:
# Create a simple function to return a tuple (image, label)
def get_image_label_2(image_path, label):
  """
  Takes an image file path name and the associated label,
  processes the image and returns a tuple of (image, label).
  """
  image = process_image_2(image_path)
  return image, label

In [ ]:
# Define the batch size, 32 is a good default
BATCH_SIZE=32
# Create a function to turn data into batches
def create_data_batches(x, y=None, batch_size=BATCH_SIZE, valid_data=False, test_data=False):
  """
  Creates batches of data out of image (x) and label (y) pairs.
  Shuffles the data if it's training data but doesn't shuffle it if it's validation data.
  Also accepts test data as input (no labels).
  """
  # If the data is a training dataset, we shuffle it
  print("Creating training data batches...")
  # Turn filepaths and labels into Tensors
  data = tf.data.Dataset.from_tensor_slices((tf.constant(x), # filepaths
                                            tf.constant(y))) # labels
  

  # Create (image, label) tuples (this also turns the image path into a preprocessed image)
  data = data.map(get_image_label)

    # Turn the data into batches
  data_batch = data.batch(BATCH_SIZE)
  return data_batch

In [ ]:
# Define the batch size, 32 is a good default
BATCH_SIZE=32
# Create a function to turn data into batches
def create_data_batches_2(x, y=None, batch_size=BATCH_SIZE, valid_data=False, test_data=False):
  """
  Creates batches of data out of image (x) and label (y) pairs.
  Shuffles the data if it's training data but doesn't shuffle it if it's validation data.
  Also accepts test data as input (no labels).
  """
  # If the data is a training dataset, we shuffle it
  print("Creating training data batches...")
  # Turn filepaths and labels into Tensors
  data = tf.data.Dataset.from_tensor_slices((tf.constant(x), # filepaths
                                            tf.constant(y))) # labels
  

  # Create (image, label) tuples (this also turns the image path into a preprocessed image)
  data = data.map(get_image_label_2)

    # Turn the data into batches
  data_batch = data.batch(BATCH_SIZE)
  return data_batch

In [ ]:
len(X)

4000

In [ ]:
# Turn full training data in a data batch

# full_data = create_data_batches(X, y, 4000)

In [ ]:
full_data_test = create_data_batches(X_test, y_test, 32)

Creating training data batches...


In [ ]:
full_data_test

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 16), dtype=tf.bool, name=None))>

In [ ]:
full_data_test_2= create_data_batches(X_test, y_test, 32)

Creating training data batches...


In [ ]:
full_data_test_2

<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 16), dtype=tf.bool, name=None))>

In [ ]:
# for image, label in full_data.map:
#   print(image.shape, label.shape)

In [ ]:
# Setup input shape to the model
INPUT_SHAPE = [None, IMG_SIZE, IMG_SIZE, 3] # batch, height, width, colour channels

# Setup output shape of the model
OUTPUT_SHAPE = len(unique_labels) # number of unique labels

# Setup model URL from TensorFlow Hub
# MODEL_URL = "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/5"
# MODEL_URL ="https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/classification/2"
MODEL_URL ="https://tfhub.dev/google/supcon/resnet_v1_200/imagenet/classification/1"


In [ ]:
# we will build the model using the Keras API

def create_model(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE, model_url=MODEL_URL):
  print("Building the model with:", MODEL_URL)

  # Setup the model layers
  model = tf.keras.Sequential([
    hub.KerasLayer(MODEL_URL), # Layer 1 (input layer)
    tf.keras.layers.Dense(units=OUTPUT_SHAPE, 
                          activation="softmax") # Layer 2 (output layer). Softmax will predict the probabilities for each class for each image
  ])

  # Compile the model
  model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(), # Our model wants to reduce this (how wrong its guesses are)
      optimizer=tf.keras.optimizers.Adam(), # An optimizer helping our model how to improve its guesses
      metrics=["accuracy"] # We'd like this to go up
  )

  # Build the model
  model.build(INPUT_SHAPE) # Let the model know what kind of inputs it'll be getting
  
  return model

## Ensemble of top models by loss and Accuracy

In [ ]:
def save_model(model, suffix=None):
  """
  Saves a given model in a models directory and appends a suffix (str)
  for clarity and reuse.
  """
  # Create model directory with current time
  modeldir = os.path.join("models_archit",
                          datetime.datetime.now().strftime("%Y%m%d-%H%M%s"))
  model_path = modeldir + "-" + suffix + ".h5" # save format of model
  print(f"Saving model to: {model_path}...")
  model.save(model_path)
  return model_path

In [ ]:
def load_model(model_path):
  """
  Loads a saved model from a specified path.
  """
  print(f"Loading saved model from: {model_path}")
  model = tf.keras.models.load_model(model_path,
                                     custom_objects={"KerasLayer":hub.KerasLayer})
  return model

In [ ]:
full_data_test

<MapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(16,), dtype=tf.bool, name=None))>

In [ ]:
!ls

'~'				        mobile_net_l.png
 Datathon2.ipynb		        models
 datathonindoml-2022.zip	        models_archit
 densenet_a.png			       'New Models.ipynb'
 dense-net-logger.csv		        predicted_label_3.csv
 densenet_l.png			        predicted_label.csv
 drive				       'rekhani LSTM.ipynb'
 efficient_net_a.png		        resnet_a.png
 efficient-net-logger.csv	        res-net-logger.csv
 efficient_net_l.png		        resnet_l.png
'few shot.ipynb'		        sample_submission.csv
'for RESNETS.ipynb'		        train
 full_np_array.npy		        train_labels.csv
 kaggle-indoml-submission.csv	        validation
 kaggle-indoml-submission-model-2.csv   vision_trans_a.png
 logs				       'Vision Transformers.ipynb'
 mobile_net_a.png		        vision-trans-logger.csv
 mobile-net-logger.csv		        vision_trans_l.png
 mobile-net-logger.gsheet


In [ ]:
# # Evaluate the loaded model for all models

# resnet_archit
loaded_model = load_model('models_archit/20221112-15151668266108-res-net.h5')
loaded_model.evaluate(full_data_test)

# dense
loaded_model = load_model('models_archit/20221112-09411668246088-dense-net.h5')
loaded_model.evaluate(full_data_test)

# mobile
loaded_model = load_model('models_archit/20221112-00031668211416-mobile-net.h5')
loaded_model.evaluate(full_data_test)

# mobile_raj
loaded_model = load_model('models/20221007-04581665118720-initial-4000-images-Adam.h5')
loaded_model.evaluate(full_data_test)

# mobile_raj_again
loaded_model = load_model('models/20221007-08281665131319-full-trained-adam.h5')
loaded_model.evaluate(full_data_test)

# mobile_raj_again_again
loaded_model = load_model('models/20221008-14551665240924-full-model-2-Adam.h5')
loaded_model.evaluate(full_data_test)
predictions = loaded_model.predict(full_data_test, verbose=1) 
predictions=pd.DataFrame(predictions)
predictions.to_csv('final_submit_approx_predicted_label_mobile-net-full.csv')

# efficient
loaded_model = load_model('models_archit/20221112-02251668219959-efficient-net.h5')
loaded_model.evaluate(full_data_test)
predictions = loaded_model.predict(full_data_test, verbose=1) 
predictions=pd.DataFrame(predictions)
predictions.to_csv('final_submit_approx_predicted_label_efficient-net.csv')


Loading saved model from: models_archit/20221112-15151668266108-res-net.h5
32/32 [==============================] - 15s 341ms/step - loss: 1.7276 - accuracy: 0.4530
Loading saved model from: models_archit/20221112-09411668246088-dense-net.h5
32/32 [==============================] - 8s 230ms/step - loss: 4.5251 - accuracy: 0.2220
Loading saved model from: models_archit/20221112-00031668211416-mobile-net.h5
32/32 [==============================] - 10s 284ms/step - loss: 1.2096 - accuracy: 0.6310
Loading saved model from: models/20221007-04581665118720-initial-4000-images-Adam.h5
32/32 [==============================] - 8s 230ms/step - loss: 1.5378 - accuracy: 0.5470
Loading saved model from: models/20221007-08281665131319-full-trained-adam.h5
32/32 [==============================] - 7s 216ms/step - loss: 0.9350 - accuracy: 0.7040
Loading saved model from: models/20221008-14551665240924-full-model-2-Adam.h5
32/32 [==============================] - 7s 214ms/step
Loading saved model from: m

In [ ]:
# trans
loaded_model = load_model('models_archit/20221112-20131668284006-vision-trans.h5')
loaded_model.evaluate(full_data_test_2)

predictions = loaded_model.predict(full_data_test_2, verbose=1) 
predictions=pd.DataFrame(predictions)
predictions.to_csv('final_submit_approx_predicted_label_vision_trans.csv')

Loading saved model from: models_archit/20221112-20131668284006-vision-trans.h5
32/32 [==============================] - 9s 239ms/step


In [ ]:

p1=pd.read_csv('final_submit_approx_predicted_label_efficient-net.csv').drop(['Unnamed: 0'],axis=1)
p2=pd.read_csv('final_submit_approx_predicted_label_mobile-net-full.csv').drop(['Unnamed: 0'],axis=1)
p3=pd.read_csv('final_submit_approx_predicted_label_vision_trans.csv').drop(['Unnamed: 0'],axis=1)


In [ ]:
p_loss=p1*0.6060+p2*0.7200+p3*0.7520/(0.6060+0.7200+0.7520)
p_acc=(p1*(1/1.2357)+p2*(1/0.8910)+p3*(1/1.5112))/((1/1.2357)+(1/0.8910)+(1/1.5112))

In [ ]:
p_loss
p_loss.to_csv('ensemble_loss.csv')

In [ ]:
loss=tf.keras.losses.CategoricalCrossentropy()

In [ ]:
p_acc
p_acc.to_csv('ensemble_accuracy.csv')